# Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, Input, MaxPool2D
from tqdm import tqdm

# Data Loading

In [ ]:
def load_data(folder):
    images = []
    for file in os.listdir(folder):
        file_id = file.replace('.png', '')
        image = Image.open(
            os.path.join(folder, file)
        ).convert('LA').resize((256, 256))
        arr = np.array(image)
        images.append(
            (int(file_id), arr)
        )
    images.sort(key=lambda i: i[0])
    return np.array([v for _id, v in images])


x_train = load_data('drive/My Drive/873/train')
y_train = pd.read_csv('drive/My Drive/873/y_train.csv')['infection']

In [ ]:
x_train.shape

# Models

*Note: if a layer is used in multiple models, it is only explained in the comments the first time it appears.*

## Fully Connected

In [ ]:
# Fully connected model 1 (given in example code)
def build_FC1():
    # Tensor object for images
    img_in = Input(shape=(256, 256, 2)) # [None, 256, 256, 2]
    # Flattens tensor to vector
    flattened = Flatten()(img_in) # [None, 131,072]
    # Fully connected layer with 64 units
    x = Dense(64)(flattened) # [None, 64]
    # Fully connected layer with 32 units
    x = Dense(32)(x) # [None, 32]
    
    # Output layer, sigmoid activation
    output = Dense(1, activation = 'sigmoid')(x) # [None, 1]
    # Create the model
    model = Model(inputs=img_in, outputs=output, name="fc1")
    return model


model = build_FC1()
# Compile model with
#   - Adam optimizer
#   - Binary Cross entropy loss - used for binary classification
#   - Binary accuracy and AUC evaluation metrics
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['BinaryAccuracy', 'AUC']
        )

# model.summary()

In [ ]:
# Fully connected model 2 (given in example code)
# Using dropout layers => randomly set input units to 0 at each step during training
# Version 1: Input -> Flatten -> Dense64 -> Dropout(0.3) -> Dense32 -> Dropout(0.3) -> Ouput
# Version 2: increase dropout rate to 0.5
# Version 3: increase dropout rate to 0.8 
def build_FC2():
    img_in = Input(shape=(256, 256, 2)) # [None, 256, 256, 2]
    flattened = Flatten()(img_in) # [None, 131072]
    x = Dense(64)(flattened) # [None, 64]
    # Dropout layer with 0.8 rate
    x = Dropout(0.8)(x) # [None, 64]
    x = Dense(32)(x) # [None, 32]
    # Dropout layer with 0.8 rate
    x = Dropout(0.8)(x) # [None, 32]
    
    output = Dense(1, activation = 'sigmoid')(x) # [None, 1]
    model = Model(inputs=img_in, outputs=output, name="fc2_3")
    return model

model = build_FC2()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        # Evaluating matches with two metrics
        metrics=['BinaryAccuracy', 'AUC']
        )

# model.summary()

In [ ]:
# Fully connected model 3
# Exploring adding layers
# Version 1: adding a Dense16 layer after the 32 and another dropout
#  Input -> Flatten -> Dense64 -> Dropout -> Dense32 -> Dropout -> Dense16 -> Dropout -> Output 
# Version 2: increase dropout to 0.8 
# Version 3: moved extra layer to the front, made it 128 nodes, back to 0.5 dropout
# Version 4: adding another layer to the front, doubling size again
def build_FC3():
    img_in = Input(shape=(256, 256, 2)) # [None, 256, 256, 2]
    flattened = Flatten()(img_in) # [None, 131072]
    # Added next dense and dropout in version 4
    x = Dense(256)(flattened) # [None, 256]
    x = Dropout(0.5)(x) # [None, 256]
    # Added next dense and dropout in version 3
    x = Dense(128)(x) # [None, 128]
    x = Dropout(0.5)(x) # [None, 128]
    
    x = Dense(64)(x) # [None, 64]
    # Changed from 0.5 to 0.8 in version 2 and back to 0.5 in version 3
    x = Dropout(0.5)(x) # [None, 64]
    x = Dense(32)(x) # [None, 32]
    # Changed from 0.5 to 0.8 in version 2 and back to 0.5 in version 3
    x = Dropout(0.5)(x) # [None, 32]

    # Added in version 1, removed in version 3
    # x = Dense(16)(x) # [None, 16]
    # Changed from 0.5 to 0.8 in version 2
    # x = Dropout(0.5)(x) # [None, 16]

    output = Dense(1, activation = 'sigmoid')(x) # [None, 1]
    model = Model(inputs=img_in, outputs=output, name="fc3_5")
    return model

model = build_FC3()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        # Evaluating matches with two metrics
        metrics=['BinaryAccuracy', 'AUC']
        )

# model.summary()

In [ ]:
# Fully connected model 4
# Exploring activation functions
# Version 1: using softmax on all hidden layers
# Version 2: tanh activation on all hidden layers
# Version 3: relu activation on all hidden layers
# Version 4: relu, no dropout
def build_FC4():
    img_in = Input(shape=(256, 256, 2)) # [None, 256, 256, 2]
    flattened = Flatten()(img_in) # [None, 131072]

    x = Dense(128, activation='relu')(flattened) # [None, 128]
    # Removed in version 4
    # x = Dropout(0.5)(x) # [None, 128]
    x = Dense(64, activation='relu')(x) # [None, 64]
    # Removed in version 4
    # x = Dropout(0.5)(x) # [None, 64]
    x = Dense(32, activation='relu')(x) # [None, 32]
    # Removed in version 4
    # x = Dropout(0.5)(x) # [None, 32]

    output = Dense(1, activation = 'sigmoid')(x) # [None, 1]
    model = Model(inputs=img_in, outputs=output, name="fc4_4")
    return model

model = build_FC4()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        # Evaluating matches with two metrics
        metrics=['BinaryAccuracy', 'AUC']
        )

# model.summary()

In [ ]:
# Fully connected model 5
# Using fully connected model (fc3_3) with highest test score so far to test out 
# different optimizers
# Version 1: RMS Prop
# Version 2: Adagrad with learning rate = 0.001
# Version 3: Adagrad with learning rate = 0.0001 
def build_FC5():
    img_in = Input(shape=(256, 256, 2)) # [None, 256, 256, 2]

    flattened = Flatten()(img_in) # [None, 131072]
    x = Dense(128)(flattened) # [None, 128]
    x = Dropout(0.5)(x) # [None, 128]
    x = Dense(64)(x) # [None, 64]
    x = Dropout(0.5)(x) # [None, 64]
    x = Dense(32)(x) # [None, 32]
    x = Dropout(0.5)(x) # [None, 32]

    output = Dense(1, activation = 'sigmoid')(x) # [None, 1]
    model = Model(inputs=img_in, outputs=output, name="fc5_3")
    return model

model = build_FC5()
model.compile(
        # optimizer = tf.keras.optimizers.RMSprop(),
        optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.0001),
        loss='binary_crossentropy',
        # Evaluating matches with two metrics
        metrics=['BinaryAccuracy', 'AUC']
        )

# model.summary()

## Convolutional

In [ ]:
# Convolutional model 1
# Version 1: Single conv2 and pool layer, padding = 'valid'
# Version 2: Increasing number of filters to be conv8
# Version 3: conv16
# Version 4: conv8, padding = 'same'
# Version 5: kernel_size = 5
def build_conv1():
    # Tensor object for images
    img_in = Input(shape=(256,256,2)) # [None, 256, 256, 2]

    # Convolutional layer, 8 filters of size 5x5, with padding to make output 
    # the same size as input, using rectified linear unit activation function
    # which takes the max(x, 0)
    x = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same')(img_in) # [None, 256, 256, 8]
    # Max pooling layer, taking max value over 2x2 window 
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 128, 128, 8]

    # Flattens tensor to vector
    x = Flatten()(x) # [None, 131072]
    output = Dense(1, activation='sigmoid')(x) # [None, 1]

    model = Model(inputs=img_in, outputs=output, name="conv1_5")
    return model

model = build_conv1()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['BinaryAccuracy', 'AUC']
)

# model.summary()

In [ ]:
# Convolutional model 2
# Adding more layers
# Version 1: Adding another identical conv8 and pool layer
# Version 2: Making first layer have more filters => conv16
# Version 3: Added conv32 + pool
# Version 4: Added conv64 + pool
# Version 5: Added conv128 + pool
def build_conv2():
    img_in = Input(shape=(256,256,2)) # [None, 256, 256, 2]
    x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(img_in) # [None, 256, 256, 128]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 128, 128, 128]

    x = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) # [None, 128, 128, 64]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 64, 64, 64]

    x = Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(x) # [None, 64, 64, 32]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 32, 32, 32]

    x = Conv2D(filters=16, kernel_size=3, activation='relu', padding='same')(x) # [None, 32, 32, 16]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 16, 16, 16]

    x = Conv2D(filters=8, kernel_size=3, activation='relu', padding='same')(x) # [None, 16, 16, 8]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 8, 8, 8]

    # Flattens tensor to vector
    x = Flatten()(x) # [None, 512]
    output = Dense(1, activation='sigmoid')(x) # [None, 1]

    model = Model(inputs=img_in, outputs=output, name="conv2_5")
    return model

model = build_conv2()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['BinaryAccuracy', 'AUC']
)

# model.summary()

In [ ]:
# Convolutional model 3
# Trying different layer arrangement
# Version 1: Rearragned layers to be CONV => CONV => POOL => CONV => CONV => POOL 
def build_conv3():
    img_in = Input(shape=(256,256,2)) # [None, 256, 256, 2]

    x = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(img_in) # [None, 256, 256, 64]
    x = Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(x) # [None, 256, 256, 32]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 128, 128, 32]

    x = Conv2D(filters=16, kernel_size=3, activation='relu', padding='same')(x) # [None, 128, 128, 16]
    x = Conv2D(filters=8, kernel_size=3, activation='relu', padding='same')(x) # [None, 128, 128, 8]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 64, 64, 8]

    # Flattens tensor to vector
    x = Flatten()(x) # [None, 32768]
    output = Dense(1, activation='sigmoid')(x) # [None, 1]

    model = Model(inputs=img_in, outputs=output, name="conv3_1")
    return model

model = build_conv3()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['BinaryAccuracy', 'AUC']
)

# model.summary()

In [ ]:
# Convolutional Model 4
# Using convolutional model (conv1_4) with highest test score so far to test out 
# different optimizers
# Version 1: Uses RMS Prop
# Version 2: Adagrad with learning rate = 0.001
# Version 3: Adagrad with learning rate = 0.0001
def build_conv4():
    img_in = Input(shape=(256,256,2)) # [None, 256, 256, 2]
    x = Conv2D(filters=8, kernel_size=3, activation='relu', padding='same')(img_in) # [None, 128, 128, 8]
    x = MaxPool2D(pool_size=(2,2))(x) # [None, 128, 128, 8]
    x = Flatten()(x) # [None, 131072]
    output = Dense(1, activation='sigmoid')(x) # [None, 1]

    model = Model(inputs=img_in, outputs=output, name="conv4")
    return model

model = build_conv4()
model.compile(
        # optimizer = tf.keras.optimizers.RMSprop(),
        optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.0001),
        loss='binary_crossentropy',
        # Evaluating matches with two metrics
        metrics=['BinaryAccuracy', 'AUC']
)

# model.summary()

# Training

In [ ]:
# MODEL TRAINING
print("Training model: " + model.name)
# Number of forward and backward passes to perform through network
epochs = 30
# Number of training examples to use in one iteration
batch_size = 32
history = model.fit(x = x_train,
                    y = y_train,
                    batch_size = batch_size,
                    # fraction of training data hold out as validation data (30%)
                    validation_split=0.3,
                    epochs=epochs,
                    verbose=1
                    )

In [ ]:
# Create AUC metric vs. epoch plot
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('model accuracy - ' + model.name)
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Output


In [ ]:
# LOAD TEST DATA
x_test = load_data('drive/My Drive/873/test')

In [ ]:
# MODEL TESTING
y_test = model.predict(x_test)

output_name = model.name + '_' + str(epochs) + '_' + str(batch_size) + '.csv'

y_test_df = pd.DataFrame()
y_test_df['id'] = np.arange(len(y_test))
y_test_df['infection'] = y_test.astype(float)
y_test_df.to_csv(output_name, index=False)
